In [1]:
from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

In [2]:
local_llm = "llama3.2:latest"
#url_llm = "http://172.17.30.133:11434" # ordenados con 2 gpus
#url_llm = "http://172.17.30.172:11434" #COMPUTACION
url_llm = "http://localhost:11434"

from langchain_community.llms import Ollama
llm = Ollama(model=local_llm, base_url=url_llm, temperature=0) 

C:\Users\emolt\AppData\Local\Temp\ipykernel_2556\3471671973.py:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=local_llm, base_url=url_llm, temperature=0)


In [15]:
import os
import re
from typing import List, Optional
from langchain.document_loaders.base import BaseLoader
from langchain.docstore.document import Document

class StructuredTXTDirectoryLoader(BaseLoader):
    def __init__(
        self,
        directory_path: str,
        metadata_keys: Optional[List[str]] = None,
        encoding: Optional[str] = "utf-8"
    ):
        self.directory_path = directory_path
        self.metadata_keys = metadata_keys or []
        self.encoding = encoding

    def load(self) -> List[Document]:
        docs = []

        for filename in os.listdir(self.directory_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(self.directory_path, filename)
                with open(file_path, "r", encoding=self.encoding) as f:
                    content = f.read()

                metadata = {"source": filename}

                for key in self.metadata_keys:
                    # Busca líneas como "key: valor"
                    pattern = rf"{re.escape(key)}\s*:\s*(.+)"
                    match = re.search(pattern, content, re.IGNORECASE)
                    if match:
                        metadata[key] = match.group(1).strip()

                doc = Document(page_content=content, metadata=metadata)
                docs.append(doc)

        return docs


In [ ]:
metadata_columns = [
    "row", "grant agreement", "project acronym", "organisation PIC", "organisation name",
    "Small or Medium-sized Enterprise", "organisation activity type", "organisation country",
    "organisation role", "funding for the organisation", "organisation total participation cost",
    "project title", "project total cost", "funding for the project", "master call",
    "subcall", "type of proposal", "project legal basis", "project topic"
]

loader = StructuredTXTDirectoryLoader(
    directory_path=r"C:\Users\emolt\OneDrive - UMH\MASTER\TFM\projects_plain_text",
    metadata_keys=metadata_columns,
    encoding="latin1"
)

raw_documents = loader.load()


In [ ]:
print(f"Número de documentos originales: {len(raw_documents)}")

In [6]:
query_1 = "What is the objective of the project with grant agreement 740934?"
query_2 = "What is the total cost of the project with the acronym HYPERGRYD?"
query_3 = "How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
query_4 = "Which organisation played the role of coordinator in the grant agreement 777998?"
query_5 = "What topic does the project with the acronym INTERRFACE belong to?"
query_6 = "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
query_7 = "What type of proposal was the grant agreement 814416?"
query_8 = "To which master call was the project with the acronym G9NIGHT submitted?"
query_9 = "To which sub call was the project titled Electron Nanocrystallography submitted?"
query_10 = "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
query_11 = "Provide the acronym of 1 project which objective is related to robotics."
query_12 = "Provide the title of 1 project which objective is related to geolocation."
query_13 = "Provide the objective of 1 project related to digital twin."
query_14 = "Provide the objective of 3 different projects related to corrosion."
query_15 = "Provide the title of 3 different projects which objective is related to offshore structures."
query_16 = "Provide the acronym of 3 different projects which objective is related to materials engineering."
query_17 = "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
query_18 = "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
query_19 = "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
query_20 = "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
query_21 = "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
query_22 = "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."
query_23 = "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."
query_24 = "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."
query_25 = "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

## EXP 7

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=800)
documents = text_splitter.split_documents(raw_documents)

In [8]:
print(f"Número de chunks generados: {len(documents)}")

Número de chunks generados: 513923


In [7]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

C:\Users\emolt\AppData\Local\Temp\ipykernel_2556\2374641863.py:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)


In [10]:
from langchain.vectorstores import FAISS
vectorstore_7_txt = FAISS.from_documents(
    documents=documents,
    embedding=emb  
)

In [11]:
vectorstore_7_txt.save_local("faiss_index_proyectos_7_txt")

## EXP 8

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=400)
documents = text_splitter.split_documents(raw_documents)

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"} # Al no tener NVIDIA es necesario cambiarlo model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {"normalize_embeddings": True}

emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [ ]:
from langchain.vectorstores import FAISS  # o Chroma, Pinecone, etc.
# Suponiendo que `documents` es una lista de Document objects con .page_content y .metadata
vectorstore_8_txt = FAISS.from_documents(
    documents=documents,
    embedding=emb  
)

In [ ]:
vectorstore_8_txt.save_local("faiss_index_proyectos_8_txt")

## EXP 9

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
documents = text_splitter.split_documents(raw_documents)

In [16]:
for doc in documents:
    doc.page_content = f"passage: {doc.page_content}"

In [10]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name_9 = "intfloat/e5-small-v2"
model_kwargs_9 = {"device": "cpu"}
encode_kwargs_9={"normalize_embeddings": True, "batch_size": 32}

emb_9 = HuggingFaceBgeEmbeddings(model_name=model_name_9,model_kwargs=model_kwargs_9,encode_kwargs=encode_kwargs_9)

C:\Users\emolt\AppData\Local\Temp\ipykernel_11668\2086778471.py:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb_9 = HuggingFaceBgeEmbeddings(model_name=model_name_9,model_kwargs=model_kwargs_9,encode_kwargs=encode_kwargs_9)


In [18]:
from langchain.vectorstores import FAISS  # o Chroma, Pinecone, etc.
# Suponiendo que `documents` es una lista de Document objects con .page_content y .metadata
vectorstore_9_txt = FAISS.from_documents(
    documents=documents,
    embedding=emb_9  
)

In [19]:
vectorstore_9_txt.save_local("faiss_index_proyectos_9_txt")

## EXP 10

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=800)
documents = text_splitter.split_documents(raw_documents)

In [8]:
for doc in documents:
    doc.page_content = f"passage: {doc.page_content}"

In [11]:
from langchain.vectorstores import FAISS  # o Chroma, Pinecone, etc.
# Suponiendo que `documents` es una lista de Document objects con .page_content y .metadata
vectorstore_10_txt = FAISS.from_documents(
    documents=documents,
    embedding=emb_9  
)

In [12]:
vectorstore_10_txt.save_local("faiss_index_proyectos_10_txt")